In [1]:
import re
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
from scipy.optimize import curve_fit

In [2]:
CPU = "M3_dit"

In [3]:
def quadratic(n, c2, c1, c0):
    return c2 * n**2 + c1 * n + c0


def subquadratic(n, c1, c0):
    return c1 * n**c0

In [4]:
n_pm = []
cycles_pm = []

with open(f"../speed_results_{CPU}/polymul_scaling.txt", "r", encoding="utf-8") as f:
    for line in f:
        match = re.match(r"amx_polymul (\d+) x (\d+): (\d+)", line)
        if match:
            n_pm.append(match.group(1))
            cycles_pm.append(match.group(3))

n_pm = np.array(n_pm, dtype=np.float64)
cycles_pm = np.array(cycles_pm, dtype=np.float64)

In [5]:
n_pmm = []
cycles_pmm = []

with open(f"../speed_results_{CPU}/polymodmul_scaling.txt", "r", encoding="utf-8") as f:
    for line in f:
        match = re.match(r"amx_polymodmul (\d+) x (\d+): (\d+)", line)
        if match:
            n_pmm.append(match.group(1))
            cycles_pmm.append(match.group(3))

n_pmm = np.array(n_pmm, dtype=np.float64)
cycles_pmm = np.array(cycles_pmm, dtype=np.float64)

In [6]:
n_pmmk = []
cycles_pmmk = []

with open(f"../speed_results_{CPU}/polymodmul_karatsuba_scaling.txt", "r", encoding="utf-8") as f:
    for line in f:
        match = re.match(r"amx_polymodmul (\d+) x (\d+): (\d+)", line)
        if match:
            n_pmmk.append(match.group(1))
            cycles_pmmk.append(match.group(3))

n_pmmk = np.array(n_pmmk, dtype=np.float64)
cycles_pmmk = np.array(cycles_pmmk, dtype=np.float64)

In [7]:
indices_pm = np.where(n_pm <= 832)[0]
indices_pmm = np.where(n_pmm <= 832)[0]

In [8]:
quadratic_fit_pm, _ = curve_fit(quadratic, n_pm[indices_pm], cycles_pm[indices_pm])
subquadratic_fit_pm, _ = curve_fit(subquadratic, n_pm[indices_pm], cycles_pm[indices_pm])

In [9]:
quadratic_fit_pmm, _ = curve_fit(quadratic, n_pmm[indices_pmm], cycles_pmm[indices_pmm])
subquadratic_fit_pmm, _ = curve_fit(subquadratic, n_pmm[indices_pmm], cycles_pmm[indices_pmm])

In [10]:
matplotlib.use("pgf")
matplotlib.rcParams.update({
    "pgf.texsystem": "pdflatex",
    'font.family': 'serif',
    'text.usetex': True,
    'pgf.rcfonts': False,
})

In [11]:
fig, axs = plt.subplots(1, 2, figsize=(10, 2))

axs[0].plot(n_pm[indices_pm], cycles_pm[indices_pm], "kx", label="w/o reduction")
# plt.plot(n_pm, quadratic(n_pm, *quadratic_fit_pm), "r-", label="fit: %f*x^2 + %f*x + %f" % tuple(quadratic_fit_pm))
axs[0].plot(
    n_pm[indices_pm],
    subquadratic(n_pm[indices_pm], *subquadratic_fit_pm),
    "k-",
    label=r"fit: $%.3f x^{%.3f}$" % tuple(subquadratic_fit_pm)
)
axs[0].plot(n_pmm[indices_pmm], cycles_pmm[indices_pmm], "ko", label="with reduction")
# plt.plot(n_pmm, quadratic(n_pmm, *quadratic_fit_pmm), "y-", label="fit: %f*x^2 + %f*x + %f" % tuple(quadratic_fit_pmm))
axs[0].plot(
    n_pmm[indices_pmm],
    subquadratic(n_pmm[indices_pmm], *subquadratic_fit_pmm),
    "k:",
    label=r"fit: $%.3f x^{%.3f}$" % tuple(subquadratic_fit_pmm)
)
axs[0].set_xlabel("$n$")
axs[0].set_ylabel("cycles")
axs[0].legend(fontsize='large')
# axs[0].legend()
axs[0].grid(linestyle = ':', linewidth = 0.5)
axs[0].set_xticks(np.arange(0, 900, 100))
axs[0].set_yticks(np.arange(0, 7001, 1000))
axs[0].set_ylim(0, 7500)
axs[0].set_xlim(-50, 900)
axs[0].text(-0.08, -0.14, "(a)", transform=axs[0].transAxes, size=12)


axs[1].plot(n_pmm, cycles_pmm, "k", label="Schoolbook with reduction")
axs[1].plot(n_pmmk, cycles_pmmk, "k--", label="Karatsuba with reduction")
axs[1].set_xlabel("$n$")
axs[1].set_ylabel("cycles")
axs[1].legend(fontsize='large')
# axs[1].legend()
axs[1].grid(linestyle = ':', linewidth = 0.5)
axs[1].set_xticks(np.arange(0, 8200, 1000))
axs[1].set_yticks(np.arange(0, 300000, 50000))
axs[1].set_ylim(0, 300000)
axs[1].text(-0.08, -0.14, "(b)", transform=axs[1].transAxes, size=12)

# fig.show()

plt.savefig(f"../speed_results_{CPU}/polymul_scaling.pgf", bbox_inches='tight', pad_inches=0)